## Now we move onto coding the ML model itself

First we make a manifest of cleaned frames, this helps keep everything organised in one place and makes the whole process easier as we dont need to continuously scan our folders every time. The function below scans `frames_3fps_clean` folder and collects the image paths and agent names into a dataframe called `manifest_df`.

In [4]:
from pathlib import Path
import pandas as pd

PROJECT_DIR   = Path(r"D:\Valorant ML data")
FRAME_OUT_DIR = PROJECT_DIR / "frames_3fps_clean"

print("Using frames from:", FRAME_OUT_DIR)

Using frames from: D:\Valorant ML data\frames_3fps_clean


In [5]:
def build_manifest(frames_root: Path):
    rows = []
    frames_root = Path(frames_root)
    for agent_folder in sorted(frames_root.iterdir()):
        if agent_folder.is_dir():
            agent = agent_folder.name
            for img_path in sorted(agent_folder.glob("*.jpg")):
                rows.append({"filepath": str(img_path), "agent": agent})
    return pd.DataFrame(rows)

manifest_df = build_manifest(FRAME_OUT_DIR)

print("total images:", len(manifest_df))
print("\nper-agent counts:")
print(manifest_df["agent"].value_counts().sort_index().to_string())

manifest_df.head()

total images: 4671

per-agent counts:
agent
Astra        185
Breach       241
Brimstone    305
Chamber      347
Cipher       337
Clove        324
Deadlock     299
Fade         321
Gekko        345
Iso          356
Jett         215
Kayo         359
Neon         335
Raze         341
Sage         361


,filepath,agent
0,D:\Valorant ML data\frames_3fps_clean\Astra\Va...,Astra
1,D:\Valorant ML data\frames_3fps_clean\Astra\Va...,Astra
2,D:\Valorant ML data\frames_3fps_clean\Astra\Va...,Astra
3,D:\Valorant ML data\frames_3fps_clean\Astra\Va...,Astra
4,D:\Valorant ML data\frames_3fps_clean\Astra\Va...,Astra


Now we need to create a split for the data for the training, validation and test groups. I just arbitrarily chose a simple 70/15/15 split. I also use stratification so that each agent is included in all the groups. 

In [6]:
from sklearn.model_selection import train_test_split
train_test_split?

Signature:
train_test_split(
    *arrays,
    test_size=None,
    train_size=None,
    random_state=None,
    shuffle=True,
    stratify=None,
)
Docstring:
Split arrays or matrices into random train and test subsets.

Quick utility that wraps input validation,
``next(ShuffleSplit().split(X, y))``, and application to input data
into a single call for splitting (and optionally subsampling) data into a
one-liner.

Read more in the :ref:`User Guide <cross_validation>`.

Parameters
----------
*arrays : sequence of indexables with same length / shape[0]
    Allowed inputs are lists, numpy arrays, scipy-sparse
    matrices or pandas dataframes.

test_size : float or int, default=None
    If float, should be between 0.0 and 1.0 and represent the proportion
    of the dataset to include in the test split. If int, represents the
    absolute number of test samples. If None, the value is set to the
    complement of the train size. If ``train_size`` is also None, it will
    be set to 0.25.

trai

In [7]:
# 70% train and 30% temp
train_df, temp_df = train_test_split(
    manifest_df,
    test_size=0.30,
    stratify=manifest_df['agent'],
    random_state=42)

# split the remaining 30% into 15% val, 15% test (still stratified)
val_df, test_df = train_test_split(
    temp_df,
    test_size=0.50,    
    stratify=temp_df['agent'],
    random_state=42
)

print("split sizes:  ", "train:", len(train_df), "| val:", len(val_df), "| test:", len(test_df))

# quick look at class balance in the train split
print("\ntrain counts:")
print(train_df['agent'].value_counts().sort_index().to_string())

split sizes:   train: 3269 | val: 701 | test: 701

train counts:
agent
Astra        130
Breach       169
Brimstone    213
Chamber      243
Cipher       236
Clove        227
Deadlock     209
Fade         225
Gekko        241
Iso          249
Jett         150
Kayo         251
Neon         234
Raze         239
Sage         253


There is quite a bit of variance here (I admit my data collection was not the best). Out of interest, I calculate summary stats of interest:

In [8]:
train_counts = train_df['agent'].value_counts().sort_index()

mean_count = train_counts.mean()
var_count  = train_counts.var()  

print("Mean:", round(mean_count, 2))
print("Variance:", round(var_count, 2))
print("Standard Deviation:", round(var_count**0.5, 2))

Mean: 217.93
Variance: 1461.07
Standard Deviation: 38.22


We now set up basic torchvision transforms:

In [9]:
from torchvision import transforms

# plain list of classes in a fixed order
class_names = sorted(train_df['agent'].unique())
num_classes = len(class_names)

print("classes:", class_names)
print("num_classes:", num_classes)

# we then define variables for image transforms 
IMG_SIZE = 224
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

train_tfms = transforms.Compose([transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),])

eval_tfms = transforms.Compose([transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),])

classes: ['Astra', 'Breach', 'Brimstone', 'Chamber', 'Cipher', 'Clove', 'Deadlock', 'Fade', 'Gekko', 'Iso', 'Jett', 'Kayo', 'Neon', 'Raze', 'Sage']
num_classes: 15


We then create a small data set that takes image paths from the pandas data frame and :

converts from BGR to RGB

applies our transforms

and returns a tuple `(image_tensor, label_int)`.

Then we build train/val/test loaders.



In [10]:
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image

class FrameTableDataset(Dataset):
    def __init__(self, table, class_names, transform=None):
        self.table = table.reset_index(drop=True) #this makes the indexing clean
        self.class_names = list(class_names)
        self.transform = transform

    def __len__(self):
        return len(self.table)  #this returns no. of rows in the table

    def __getitem__(self, idx):
        row = self.table.iloc[idx]
        path = row['filepath']
        label_name = row['agent']

        # map label name to integer
        label = self.class_names.index(label_name)

        img_bgr = cv2.imread(path)
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(img_rgb)

        if self.transform is not None:
            x = self.transform(pil_img)
        else:
            x = torch.from_numpy(np.transpose(img_rgb, (2, 0, 1))).float() / 255.0
        return x, int(label)

# datasets
train_ds = FrameTableDataset(train_df, class_names, transform=train_tfms)
val_ds = FrameTableDataset(val_df, class_names, transform=eval_tfms)
test_ds = FrameTableDataset(test_df, class_names, transform=eval_tfms)

#dataloaders
BATCH_SIZE = 32
NUM_WORKERS = 0
PIN_MEMORY = True

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
val_loader = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
test_loader = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)

print("sizes:", "train:", len(train_ds), "| val:", len(val_ds), "| test:", len(test_ds))

sizes: train: 3269 | val: 701 | test: 701


Here I choose my device. Because I have an RTX 3060 Ti I obviously want to use cuda to speed up the training process. I also load the pretrained resnet18, however I need to adjust the final output layer to the number of agents (15).

In [11]:
import torch
import torch.nn as nn
from torchvision import models
DEVICE = torch.device("cuda")

num_classes = len(class_names)

model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
in_feats = model.fc.in_features
model.fc = nn.Linear(in_feats, num_classes) #this adjust the final layer

# move to device, which is 3060 ti in my case
model = model.to(DEVICE)

print("device:", DEVICE)

device: cuda


WE will run gradient descent with Adam. cross-entropy loss matches multi-class classification by maxxing the log likelihood of the correct class. We print average loss (how wrong on average) and accuracy each epoch. Online its recommended to use a small learning rate specifically when tuning a pre trained net.
On the forward pass we `model(x)` calculateds raw class scores, `criterion(logits, y)` calculates the cross-entropy loss. Then on the backprop `loss.backward()` computes gradients, and `optimiser.step()` uses ADAM to update the weights.. 

After training I went back to also save important values, which will be used for grapnhing later (I wish I did this before I trained, so I wouldnt have to wait for model to re-train).

In [12]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimiser = optim.Adam(model.parameters(), lr=1e-4)

EPOCHS = 5

In [13]:
import torch

def train_one_epoch(model, loader, optimiser, device):
    model.train()   #this puts the layers in training mode
    total_loss = 0.0
    total_correct = 0
    total_seen = 0   #we set all this counters to 0 at first

    for x, y in loader:  #here x=images and y=labels
        x = x.to(device)
        y = y.to(device)  #offloads data to 3060 Ti for training

        optimiser.zero_grad()  #clears old gradients to prevent them building up across batches
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimiser.step()

        total_loss += loss.item() * x.size(0)     #we add this batchs loss to the running total, we multiply by batch size
        preds = logits.argmax(dim=1)       #the predicted class=index of the largest score per sample
        total_correct += (preds == y).sum().item()    #this counts how many predictions match the true labels (y) in this batch
        total_seen += x.size(0)     #this is just a counter of batches.

    avg_loss = total_loss / max(1, total_seen)    #need the max functon here or else could divide by 0
    acc = total_correct / max(1, total_seen)
    return avg_loss, acc

In [14]:
@torch.no_grad()
#this fucntion evaluates on a loader(which is the val set here)
def eval_one_loader(model, loader, device):
    model.eval()   #eval mode
    total_loss = 0.0
    total_correct = 0
    total_seen = 0

    for x, y in loader:
        x = x.to(device)
        y = y.to(device)

        logits = model(x)     #forward pass only and no grads calculated
        loss = criterion(logits, y)       #we need to keep track of loss here as we use val loss to spot overfitting

        total_loss += loss.item() * x.size(0)
        preds = logits.argmax(dim=1)
        total_correct += (preds == y).sum().item()
        total_seen += x.size(0)

    avg_loss = total_loss / max(1, total_seen)
    acc = total_correct / max(1, total_seen)
    return avg_loss, acc

In [15]:
epoch_list = [] 
train_loss_hist = [] 
val_loss_hist = []  
train_acc_hist = [] 
val_acc_hist = []  

for epoch in range(1, EPOCHS + 1):   #1-indexxed for nicer printing
    tr_loss, tr_acc = train_one_epoch(model, train_loader, optimiser, DEVICE)
    va_loss, va_acc = eval_one_loader(model, val_loader, DEVICE)
    print(f"epoch {epoch} - train loss {tr_loss:.4f}  acc {tr_acc:.4f} --- val loss {va_loss:.4f}  acc {va_acc:.4f}")

    epoch_list.append(epoch)          
    train_loss_hist.append(tr_loss)  
    val_loss_hist.append(va_loss)     
    train_acc_hist.append(tr_acc)  
    val_acc_hist.append(va_acc)

KeyboardInterrupt: 

Next we run the model on the test set with gradients off, then uses `argmax` as the predicted class. Under cross-entropy, argmax matches the class with the highest estimated probability. The per class table helps spot any issues with specific class.

In [ ]:
import torch
import numpy as np
import pandas as pd

model.eval()  #as we are doing inference

all_preds = []
all_labels = []

with torch.no_grad():
    for x, y in test_loader:
        x = x.to(DEVICE)
        y = y.to(DEVICE)

        scores = model(x)         # raw class scores
        preds = scores.argmax(1)          # pick class with highest score

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(y.cpu().numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)  #we convert these to numpy arrays 

# overall accuracy
test_acc = float((all_preds == all_labels).mean())  #we average over everyhting 
print(f"test accuracy: {test_acc:.4f}")

# per agent accuracy 
counts = {name: {"correct": 0, "total": 0} for name in class_names}
for true_lab, pred_lab in zip(all_labels, all_preds):   #iterates over tuples
    name = class_names[int(true_lab)]
    counts[name]["total"] += 1
    if pred_lab == true_lab:
        counts[name]["correct"] += 1

rows = []
for name in sorted(counts.keys()):
    cor = counts[name]["correct"]
    tot = counts[name]["total"]
    acc = (cor / tot) if tot > 0 else float("nan")  #to again prevent division by 0
    rows.append({"class": name, "correct": cor, "total": tot, "accuracy": acc})

per_class_df = pd.DataFrame(rows)

print("\nper class accuracy:")
print(per_class_df.to_string(index=False))

Okay so the heavy lifting is done, we can now do things like save our model weights to the stated file.

In [ ]:
WEIGHTS_PATH = Path(r"D:\Valorant ML data") / "resnet18_valorant_updated.pth"
torch.save(model.state_dict(), WEIGHTS_PATH)

Using matplotlib, we plot 2 graphs, one for loss and one for accuracy:

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(epoch_list, train_acc_hist, label="train acc")
plt.plot(epoch_list, val_acc_hist,   label="val acc")
plt.xlabel("epoch"); 
plt.ylabel("accuracy"); 
plt.legend(); 
plt.show()

plt.figure()
plt.plot(epoch_list, train_loss_hist, label="train loss")
plt.plot(epoch_list, val_loss_hist,   label="val loss")
plt.xlabel("epoch"); 
plt.ylabel("loss"); 
plt.legend(); 
plt.show()

Lastly, I took some extra images of different characters on a different map (Bind) to try to test the models predictive capabilities.

In [ ]:
from PIL import Image
import torch.nn.functional as F

def predict(image_path):
    model.eval()  #again we are in inference mode
    img = Image.open(image_path).convert("RGB")
    x = eval_tfms(img).unsqueeze(0).to(DEVICE)  #the shape is  1 x C x H x W
    with torch.no_grad():
        scores = model(x)    # computes class scores
        pred_idx = int(scores.argmax(1).item()) # highest score will obviously be predicted class
        prob = float(F.softmax(scores, dim=1)[0, pred_idx].item())
    plt.imshow(img)
    plt.title(f"{class_names[pred_idx]}  (p={prob:.5f})")
    plt.axis("off")
    plt.show()

In [ ]:
predict(r"D:\Valorant ML data\Screenshot (12).png")